In [1]:
import os

In [2]:
%pwd

'd:\\End-to-end-Machine-Learning-Project\\research'

In [3]:
# To move to project folder
os.chdir("../")

In [4]:
%pwd

'd:\\End-to-end-Machine-Learning-Project'

In [ ]:
# Updating the entity item in workflow list
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

    # def __post_init__(self):
    #     # Ensure that the paths are absolute
    #     self.root_dir = Path(self.root_dir).resolve()
    #     self.local_data_file = Path(self.local_data_file).resolve()
    #     self.unzip_dir = Path(self.unzip_dir).resolve()

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [8]:
# Updating configuration mgr in src
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [ ]:
# Updating the components item in workflow list
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [10]:
# Function for downloading and extracting data
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

# To extract the zip file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
# Updating the pipeline item on the workflow list
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-01 14:25:23,712: INFO: common: YAML file config\config.yaml loaded successfully.]
[2025-07-01 14:25:23,719: INFO: common: YAML file params.yaml loaded successfully.]
[2025-07-01 14:25:23,726: INFO: common: YAML file schema.yaml loaded successfully.]
[2025-07-01 14:25:23,730: INFO: common: Created directory: artifacts]
[2025-07-01 14:25:23,738: INFO: common: Created directory: artifacts/data_ingestion]


[2025-07-01 14:25:26,442: INFO: 2928369538: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 87C9:1E80B3:E8FB:31C23:6863C5A5
Accept-Ranges: bytes
Date: Tue, 01 Jul 2025 11:25:25 GMT
Via: 1.1 varnish
X-Served-By: cache-jnb7023-JNB
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1751369126.621623,VS0,VE262
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: e4ed42c616de92d472e35e6e702b25ec87fd09ed
Expires: Tue, 01 Jul 2025 11:30:25 GMT
Source-Age: 0

]
